В этом ноутбуке будет реализована классификация текстов с использованием предобученной модели BERT

In [ ]:
import json
# do not change the code in the block below
# __________start of block__________
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

%matplotlib inline
# __________end of block__________

In [ ]:
# do not change the code in the block below
# __________start of block__________

!wget https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
# __________end of block__________

--2025-09-17 13:24:37--  https://raw.githubusercontent.com/girafe-ai/ml-course/refs/heads/24f_yandex_ml_trainings/homeworks/hw04_bert_and_co/texts_holdout.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51581 (50K) [text/plain]
Saving to: ‘texts_holdout.json’

texts_holdout.json  100%[===================>]  50.37K  --.-KB/s    in 0.001s  

2025-09-17 13:24:37 (76.5 MB/s) - ‘texts_holdout.json’ saved [51581/51581]



In [ ]:
# do not change the code in the block below
# __________start of block__________
df = pd.read_csv(
    "https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv",
    delimiter="\t",
    header=None,
)
texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]
texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]
with open("texts_holdout.json") as iofile:
    texts_holdout = json.load(iofile)
# __________end of block__________

в качестве bert я выбрал deberta-v3-base-imdb - это классический microsoft/deberta-v3-base, дообученный на imdb dataset, так как у нас близская задача

In [ ]:
!pip install -Uqq transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline("text-classification", "dfurman/deberta-v3-base-imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
from transformers import AutoModel, AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('dfurman/deberta-v3-base-imdb')
model=AutoModel.from_pretrained('dfurman/deberta-v3-base-imdb')

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
model.to(device)

DebertaV2Model(
  (embeddings): DebertaV2Embeddings(
    (word_embeddings): Embedding(128100, 768, padding_idx=0)
    (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): DebertaV2Encoder(
    (layer): ModuleList(
      (0-11): 12 x DebertaV2Layer(
        (attention): DebertaV2Attention(
          (self): DisentangledSelfAttention(
            (query_proj): Linear(in_features=768, out_features=768, bias=True)
            (key_proj): Linear(in_features=768, out_features=768, bias=True)
            (value_proj): Linear(in_features=768, out_features=768, bias=True)
            (pos_dropout): Dropout(p=0.1, inplace=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): DebertaV2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

tokenize texts

In [ ]:
texts_train_tokenize = tokenizer(list(texts_train),return_tensors='pt', padding=True)
texts_test_tokenize = tokenizer(list(texts_test), return_tensors='pt',padding=True)

In [ ]:
texts_train_tokenize["input_ids"]

tensor([[    1,   266, 11684,  ...,     0,     0,     0],
        [    1,  3900, 86721,  ...,     0,     0,     0],
        [    1,   306, 22069,  ...,     0,     0,     0],
        ...,
        [    1, 20782, 26024,  ...,     0,     0,     0],
        [    1,   278,   490,  ...,     0,     0,     0],
        [    1,   266, 16883,  ...,     0,     0,     0]])

In [ ]:
len(texts_train_tokenize["input_ids"])

5000

In [ ]:
print(device)

cuda:0


сохраним эмбеддинги в features

In [ ]:
batch_size=64
features=[]

with torch.no_grad():
  for i in range (0, len(texts_train),batch_size):
    texts_batch = texts_train_tokenize["input_ids"][i:i+batch_size].to(device)
    masks_batch=texts_train_tokenize["attention_mask"][i:i+batch_size].to(device)
    output=model(texts_batch, masks_batch)
    batch_featurs=output.last_hidden_state[:,0,:].cpu().numpy()
    features.append(batch_featurs)
    if i%100==0:
      print(i)


0
1600
3200
4800


In [ ]:
features = np.concatenate(features, axis=0)
features.shape

(5000, 768)

In [ ]:
from sklearn.linear_model import LogisticRegression

lc_clf=LogisticRegression().fit(features,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


протестируем

In [ ]:
features_test=[]
batch_size=1
with torch.no_grad():
  for i in range (0, len(texts_test),batch_size):
    texts_batch = texts_test_tokenize["input_ids"][i:i+batch_size].to(device)
    masks_batch=texts_test_tokenize["attention_mask"][i:i+batch_size].to(device)
    output=model(texts_batch, masks_batch)
    batch_featurs=output.last_hidden_state[:,0,:].cpu().numpy()
    features_test.append(batch_featurs)
    if i%1000==0:
      print(i)

0
1000


In [ ]:
#

In [ ]:
features_test = np.concatenate(features_test, axis=0)
features_test.shape

(1920, 768)

In [ ]:
lc_clf.score(features_test, y_test)

0.8979166666666667

Теперь делаем submission

In [ ]:
texts_holdout_tokenize = tokenizer(list(texts_holdout), return_tensors='pt',padding=True)

In [ ]:
features_holdout=[]
batch_size=1
with torch.no_grad():
  for i in range (0, len(texts_holdout),batch_size):
    texts_batch = texts_holdout_tokenize["input_ids"][i:i+batch_size].to(device)
    masks_batch=texts_holdout_tokenize["attention_mask"][i:i+batch_size].to(device)
    output=model(texts_batch, masks_batch)
    batch_featurs=output.last_hidden_state[:,0,:].cpu().numpy()
    features_holdout.append(batch_featurs)
    if i%100==0:
      print(i)

0
100
200
300
400


In [ ]:
features_holdout = np.concatenate(features_holdout, axis=0)
features_holdout.shape

(500, 768)

In [ ]:
out_dict = {
    'train': lc_clf.predict_proba(features)[:, 1],
    'test': lc_clf.predict_proba(features_test)[:, 1],
    'holdout': lc_clf.predict_proba(features_holdout)[:, 1]
}


In [ ]:
out_dict["train"]=[float(x) for x in out_dict["train"]]

In [ ]:
out_dict["test"]=[float(x) for x in out_dict["test"]]

In [ ]:
out_dict["holdout"]=[float(x) for x in out_dict["holdout"]]

In [ ]:
assert isinstance(out_dict["train"], list), "Object must be a list of floats"
assert isinstance(out_dict["train"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["train"]) == 5000
), "The predicted probas list length does not match the train set size"

assert isinstance(out_dict["test"], list), "Object must be a list of floats"
assert isinstance(out_dict["test"][0], float), "Object must be a list of floats"
assert (
    len(out_dict["test"]) == 1920
), "The predicted probas list length does not match the test set size"

assert isinstance(out_dict["holdout"], list), "Object must be a list of floats"
assert isinstance(out_dict["holdout"][0], float), "Object must be a list of floats"
assert (len(
    out_dict["holdout"]) == 500
), "The predicted probas list length does not match the holdout set size"

In [ ]:
# do not change the code in the block below
# __________start of block__________
FILENAME = "submission_dict_hw_text_classification_with_bert.json"

with open(FILENAME, "w") as iofile:
    json.dump(out_dict, iofile)
print(f"File saved to `{FILENAME}`")
# __________end of block__________

File saved to `submission_dict_hw_text_classification_with_bert.json`
